# Amazon SageMaker Ground Truth Demonstration for Image Classification

1. [Introduction](#Introduction)
2. [Run a Ground Truth labeling job](#Run-a-Ground-Truth-labeling-job)
    1. [Prepare the data](#Prepare-the-data)
    2. [Prepare labeling input manifest file](#Prepare-labeling-input-manifest-file)
    3. [Specify Label categories](#Specify-Labels-Categories)
    4. [Create the instruction template](#Create-A-Worker-Task-Template)
    5. [Specify Parameters for Labeling Job](#Use-the-CreateLabelingJob-API-JOB-1)

## Introduction


This sample notebook demonstrates the functionality Image Classification in SageMaker Ground Truth. Before you begin, we highly recommend you start a Ground Truth labeling job through the AWS Console first to familiarize yourself with the workflow. The AWS Console offers less flexibility than the API, but is simple to use. 

### Get latest version of AWS python SDK

In [ ]:
# !pip install -q --upgrade pip
# !pip install awscli -q --upgrade
# !pip install botocore -q --upgrade
# !pip install boto3 -q --upgrade
# !pip install sagemaker -q --upgrade

# NOTE: Restart Kernel after the above command

In [ ]:
import boto3
import botocore
import json
import time
import sagemaker
import re
import os
import s3fs
import pandas as pd

### Prerequisites

You will create some of the resources you need to launch a Ground Truth streaming labeling job in this notebook. 

A work team - A work team is a group of workers that complete labeling tasks. If you want to preview the worker UI and execute the labeling task you will need to create a private work team, add yourself as a worker to this team, and provide the work team ARN below. 

In [ ]:
WORKTEAM_ARN = "<<ADD WORK TEAM ARN HERE>>"

print(f"This notebook will use the work team ARN: {WORKTEAM_ARN}")

In [ ]:
# Make sure workteam arn is populated if private work team is chosen
assert WORKTEAM_ARN != "<<ADD WORK TEAM ARN HERE>>"

* The IAM execution role you used to create this notebook instance must have the following permissions: 
    * AWS managed policy [AmazonSageMakerGroundTruthExecution](https://console.aws.amazon.com/iam/home#policies/arn:aws:iam::aws:policy/AmazonSageMakerGroundTruthExecution). Run the following code-block to see your IAM execution role name. This [GIF](add-policy.gif) demonstrates how to add this policy to an IAM role in the IAM console. You can also find instructions in the IAM User Guide: [Adding and removing IAM identity permissions](https://docs.aws.amazon.com/IAM/latest/UserGuide/access_policies_manage-attach-detach.html#add-policies-console).
    * When you create your role, you specify Amazon S3 permissions. Make sure that your IAM role has access to the S3 bucket that you plan to use in this example. If you do not specify an S3 bucket in this notebook, the default bucket in the AWS region you are running this notebook instance will be used. If you do not require granular permissions, you can attach [AmazonS3FullAccess](https://console.aws.amazon.com/iam/home#policies/arn:aws:iam::aws:policy/AmazonS3FullAccess) to your role.

In [ ]:
role = sagemaker.get_execution_role()
role_name = role.split("/")[-1]
print(
    "IMPORTANT: Make sure this execution role has the AWS Managed policy AmazonGroundTruthExecution attached."
)
print("********************************************************************************")
print("The IAM execution role name:", role_name)
print("The IAM execution role ARN:", role)
print("********************************************************************************")

## Run-a-Ground-Truth-labeling-job

### Prepare-the-data

The sample images to be labeled in this tutorial are pulled from the publicly available [Caltech 101 dataset](https://data.caltech.edu/records/mzrjq-6wc02) (Li, F.-F., Andreeto, M., Ranzato, M. A., & Perona, P. (2022). Caltech 101 (Version 1.0) [Data set]. CaltechDATA), which contains pictures in 101 object categories. To minimize the cost of this tutorial, you use a sample set of 10 images, with two images from each of the following categories: airplanes, cars, ferries, helicopters, and motorbikes. But the steps to launch a labeling job for a larger dataset are the same as the ones in this tutorial. The sample set of 10 images is already available in the Amazon S3 bucket sagemaker-sample-files.

In [ ]:
import sagemaker


sess = sagemaker.Session()
bucket = sess.default_bucket()

!aws s3 sync s3://sagemaker-sample-files/datasets/image/caltech-101/inference/ s3://{bucket}/images/

print('Copy and paste the below link into a web browser to confirm the ten images were successfully uploaded to your bucket:')
print(f'https://s3.console.aws.amazon.com/s3/buckets/{bucket}/images/')

print('\nWhen prompted by Sagemaker to enter the S3 location for input datasets, you can paste in the below S3 URL')

print(f's3://{bucket}/images/')

print('\nWhen prompted by Sagemaker to Specify a new location, you can paste in the below S3 URL')

print(f's3://{bucket}/labeled-data/')

### Prepare-labeling-input-manifest-file

SageMaker Ground Truth operates using manifests. When using a modality like image classification, a single image corresponds to a single entry in a manifest and a given manifest will directly contain paths for all of the images to be labeled. To learn how to create an input manifest file, see [Use an Input Manifest File](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-input-data-input-manifest.html). 

In [ ]:
s3 = boto3.resource('s3')
INPUT_MANIFEST_S3_PREFIX = "s3://" + bucket + "/" 
print(INPUT_MANIFEST_S3_PREFIX)
INPUT_MANIFEST_FILE_NAME = "input.manifest" #Provide an Input manifest filename
INPUT_MANIFEST = INPUT_MANIFEST_S3_PREFIX + INPUT_MANIFEST_FILE_NAME
my_bucket=s3.Bucket(name=bucket)


img_list=[]
for obj in my_bucket.objects.filter(Delimiter='/', Prefix='images/'):
    img_list.append(obj.key)

Input_image_s3=[]
for i in range(1,len(img_list)):
    Input_image_s3.append(INPUT_MANIFEST_S3_PREFIX+img_list[i])

manifest_lines = [
    {
    "source-ref": image
    }
    for image in Input_image_s3
]

s3 = s3fs.S3FileSystem(anon=False)
with s3.open(f"{INPUT_MANIFEST}",'w') as f:
    f.writelines([json.dumps(m)+"\n" for m in manifest_lines])
print(f"Input manifest file created at {INPUT_MANIFEST} with {len(manifest_lines)} tasks.")

### Labeling-Job-Name

The following cells will create a name for your labeling job. This labeling job name and these topics will be used in your CreateLabelingJob request later in this notebook.

In [ ]:
# Job Name
LABELING_JOB_NAME = "GroundTruth-Image-Classification-" + str(int(time.time()))

print("Your labeling job name will be :", LABELING_JOB_NAME)

### Choose-Labeling-Job-Built-In-Task-Type

Ground Truth supports a variety of built-in task types which streamline the process of creating image, text, video, video frame, and 3D point cloud labeling jobs. The image bounding box task type will be used by default for this demonstration. 

In [ ]:
task_type = "Image Classification (Single Label)"
print(f"Your task type: {task_type}")

In [ ]:
task_type_map = {
    "Image Classification (Single Label)": "ImageMultiClass"
}

arn_region_map = {
    "us-west-2": "081040173940",
    "us-east-1": "432418664414",
    "us-east-2": "266458841044",
    "eu-west-1": "568282634449",
    "eu-west-2": "487402164563",
    "ap-northeast-1": "477331159723",
    "ap-northeast-2": "845288260483",
    "ca-central-1": "918755190332",
    "eu-central-1": "203001061592",
    "ap-south-1": "565803892007",
    "ap-southeast-1": "377565633583",
    "ap-southeast-2": "454466003867",
}

In [ ]:
region = boto3.session.Session().region_name
task_type_suffix = task_type_map[task_type]
region_account = arn_region_map[region]
PRE_HUMAN_TASK_LAMBDA = f"arn:aws:lambda:{region}:{region_account}:function:PRE-{task_type_suffix}"
POST_ANNOTATION_LAMBDA = f"arn:aws:lambda:{region}:{region_account}:function:ACS-{task_type_suffix}"
print(PRE_HUMAN_TASK_LAMBDA)
print(POST_ANNOTATION_LAMBDA)

### Specify-Labels-Categories

You specify the labels that you want workers to use to annotate your data in a label category configuration file. Workers can assign one or more attributes to annotations to give more information about that object. 

For all task types, you can use the following cell to identify the labels you use for your labeling job. To create a label category configuration file with label category attributes, see [Create a Labeling Category Configuration File with Label Category Attributes
](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-label-cat-config-attributes.html) in the Amazon SageMaker developer guide. 

In [ ]:
LABEL_CATEGORIES = ["Airplane", "Car", "Ferry", "Helicopter", "Motorbike"]

The following cell will create a label category configuration file using the labels specified above. 

**IMPORTANT**: Make sure you have added label categories above and they appear under `labels` when you run the following cell.

In [ ]:
# Specify labels and this notebook will upload and a label category configuration file to S3.
json_body = {
    "document-version": "2018-11-28",
    "labels": [{"label": label} for label in LABEL_CATEGORIES],
}
with open("class_labels.json", "w") as f:
    json.dump(json_body, f)

print("Your label category configuration file:")
print("\n", json.dumps(json_body, indent=2))

In [ ]:
s3 = boto3.client("s3")
s3.upload_file("class_labels.json", bucket, "class_labels.json")

In [ ]:
LABEL_CATEGORIES_S3_URI = f"s3://{bucket}/class_labels.json"
print(f"You should now see class_labels.json in {LABEL_CATEGORIES_S3_URI}")

### Create-A-Worker-Task-Template

Part or all of your images will be annotated by human annotators. It is essential to provide good instructions. Good instructions are:

1. Concise. We recommend limiting verbal/textual instruction to two sentences and focusing on clear visuals.
2. Visual. In the case of object detection, we recommend providing several labeled examples with different numbers of boxes.
3. When used through the AWS Console, Ground Truth helps you create the instructions using a visual wizard. When using the API, you need to create an HTML template for your instructions. 

NOTE: If you use any images in your template (as we do), they need to be publicly accessible. You can enable public access to files in your S3 bucket through the S3 Console, as described in S3 Documentation.

In [ ]:
from IPython.display import display, HTML


def make_template(save_fname="instructions.template"):
    template = r"""<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>
    <crowd-form>
      <crowd-image-classifier
        name="crowd-image-classifier"
        src="{{{{ task.input.taskObject | grant_read_access }}}}"
        header="Please select the correct category for this image"
        categories="{{{{ task.input.labels | to_json | escape }}}}"
      >
        <full-instructions header="Classification Instructions">
          <p>Read the task carefully and inspect the image.</p>
          <p>Choose the appropriate label that best suits the image.</p>
        </full-instructions>

        <short-instructions>
          <p>Read the task carefully and inspect the image.</p>
          <p>Choose the appropriate label that best suits the image.</p>
        </short-instructions>
      </crowd-image-classifier>
    </crowd-form>

    """.format()
    with open(save_fname, "w") as f:
        f.write(template)
        
make_template(save_fname="instructions.template.html")
result = s3.upload_file("instructions.template.html", bucket, "instructions.template.html")

### Specify-Parameters-for-Labeling-Job


To learn more about these parameters, use the following documentation:
* [TaskTitle](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_HumanTaskConfig.html#sagemaker-Type-HumanTaskConfig-TaskTitle)
* [TaskDescription](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_HumanTaskConfig.html#sagemaker-Type-HumanTaskConfig-TaskDescription)
* [TaskKeywords](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_HumanTaskConfig.html#sagemaker-Type-HumanTaskConfig-TaskKeywords)

In [ ]:
TASK_TITLE = "Image Classification"

TASK_DESCRIPTION = "Classify images into the defined labels"

TASK_KEYWORDS = ["Image Classification"]

In [ ]:
# The path in Amazon S3 to your worker task template or human task UI
HUMAN_UI = []

UI_TEMPLATE_S3_URI = f"s3://{bucket}/instructions.template.html"
HUMAN_UI.append(UI_TEMPLATE_S3_URI)
UI_CONFIG_PARAM = "UiTemplateS3Uri"

print(f"{UI_CONFIG_PARAM} resource that will be used: {HUMAN_UI[0]}")

In [ ]:
# If you want to store your output manifest in a different folder, provide an OUTPUT_PATH.
OUTPUT_FOLDER_PREFIX = "/gt-demo-output"
OUTPUT_BUCKET = "s3://" + bucket + OUTPUT_FOLDER_PREFIX
print("Your output data will be stored in:", OUTPUT_BUCKET)

# An IAM role with AmazonGroundTruthExecution policies attached.
# This must be the same role that you used to create this notebook instance.
ROLE_ARN = role

### Use-the-CreateLabelingJob-API-JOB-1

In [ ]:
LABEL_ATTRIBUTE_NAME = LABELING_JOB_NAME

human_task_config = {
    "PreHumanTaskLambdaArn": PRE_HUMAN_TASK_LAMBDA,
    "MaxConcurrentTaskCount": 100,  # Maximum of 100 objects will be available to the workteam at any time
    "NumberOfHumanWorkersPerDataObject": 1,  # We will obtain and consolidate 1 human annotationsfor each image.
    "TaskAvailabilityLifetimeInSeconds": 21600,  # Your workteam has 6 hours to complete all pending tasks.
    "TaskDescription": TASK_DESCRIPTION,
    "WorkteamArn": WORKTEAM_ARN,
    "AnnotationConsolidationConfig": {"AnnotationConsolidationLambdaArn": POST_ANNOTATION_LAMBDA},
    "TaskKeywords": TASK_KEYWORDS,
    "TaskTimeLimitInSeconds": 600,  # Each image must be labeled within 10 minutes.
    "TaskTitle": TASK_TITLE,
    "UiConfig": {UI_CONFIG_PARAM: HUMAN_UI[0]},
}


human_task_config["WorkteamArn"] = WORKTEAM_ARN

ground_truth_request = {
    'InputConfig':{
        'DataSource': {
            'S3DataSource': {
                'ManifestS3Uri': INPUT_MANIFEST}
        }},
    "HumanTaskConfig": human_task_config,
    "LabelAttributeName": LABEL_ATTRIBUTE_NAME,
    "LabelCategoryConfigS3Uri": LABEL_CATEGORIES_S3_URI,
    "LabelingJobName": LABELING_JOB_NAME,
    "OutputConfig": {"S3OutputPath": OUTPUT_BUCKET},
    "RoleArn": ROLE_ARN,
}

#### DataAttributes
You should not share explicit, confidential, or personal information or protected health information with the Amazon Mechanical Turk workforce. 

If you are using Amazon Mechanical Turk workforce, you must verify that your data is free of personal, confidential, and explicit content and protected health information using this code cell. 

In [ ]:
ground_truth_request["InputConfig"]["DataAttributes"] = {
    "ContentClassifiers": ["FreeOfPersonallyIdentifiableInformation", "FreeOfAdultContent"]
}

In [ ]:
print("Your create labeling job request:\n", json.dumps(ground_truth_request, indent=4))

In [ ]:
sagemaker_client = boto3.client("sagemaker")
sagemaker_client.create_labeling_job(**ground_truth_request)

### Use the DescribeLabelingJob API to describe Labeling Job

In [ ]:
sagemaker_client.describe_labeling_job(LabelingJobName=LABELING_JOB_NAME)